In [ ]:
import pandas as pd
from pathlib import Path

src = Path("../data/original/supply_chain_augmented.csv")
dst_dir = Path("../data/derived")
dst_dir.mkdir(parents=True, exist_ok=True)
dst = dst_dir / "logistics_data.csv"

df = pd.read_csv(src, parse_dates=["date"])

df["day"] = df["date"].dt.day
df["week"] = df["date"].dt.isocalendar().week.astype(int)
df["month"] = df["date"].dt.month

df["promotions"] = (df["discount_percent"] > 0).astype(int)

columns = [
    "shipment_id",
    "order_id",
    "origin_lat",
    "origin_lng",
    "destination_lat",
    "destination_lng",
    "status",
    "event_timestamp",
    "estimated_delivery_date",
    "actual_delivery_date",
    "delay_hours",
    "shipping_carrier",
    "shipping_time_days",
    "shipping_cost",
    "shipping_carrier_original",
    "shipping_cost_original",
    "location",
]

df[columns].to_csv(dst, index=False)
print(f"Saved: {dst}")

Saved: ..\data\derived\logistics_data.csv


In [6]:
import pandas as pd
import numpy as np
from pathlib import Path

# Paths
pos_path = Path("../data/original/supply_chain_pos.csv")
aug_path = Path("../data/original/supply_chain_augmented.csv")
dst_dir = Path("../data/derived")
dst_dir.mkdir(parents=True, exist_ok=True)
dst_path = dst_dir / "vendor_data.csv"

# Read base PO dataset (authoritative for PO records)
pos_cols = ["po_id", "sku_id", "vendor_id", "po_date", "po_qty", "promised_delivery_date"]
pos = pd.read_csv(pos_path, parse_dates=["po_date", "promised_delivery_date"])

# Read augmented dataset (contains vendor features)
aug = pd.read_csv(
    aug_path,
    parse_dates=["po_date", "promised_delivery_date"],
    low_memory=False,
)

# Select vendor-related attributes from augmented data (one row per po_id)
aug_vendor_cols = [
    "po_id",
    "supplier_name",
    "actual_receipt_qty",
    "inspection_results",
    "defect_rate",
    "production_volumes",
    "manufacturing_lead_time",
    "manufacturing_costs",
    "lead_time_days",
    "supplier_name_original",
    "inspection_results_original",
    "defect_rates_original",
]
# Keep only columns that exist (safety)
aug_vendor_cols = [c for c in aug_vendor_cols if c in aug.columns]
aug_vendor = aug[aug_vendor_cols].drop_duplicates(subset=["po_id"], keep="last")

# Merge: POS base + vendor attributes on po_id (join key)
df = pos.merge(aug_vendor, on="po_id", how="left")

# If lead_time_days is missing, backfill from promised_delivery_date - po_date
if "lead_time_days" not in df.columns:
    df["lead_time_days"] = (df["promised_delivery_date"] - df["po_date"]).dt.days
else:
    missing = df["lead_time_days"].isna()
    df.loc[missing, "lead_time_days"] = (
        df.loc[missing, "promised_delivery_date"] - df.loc[missing, "po_date"]
    ).dt.days

# Ensure all requested columns exist (create empty if missing)
requested_cols = [
    "vendor_id",
    "supplier_name",
    "po_id",
    "po_date",
    "po_qty",
    "promised_delivery_date",
    "actual_receipt_qty",
    "inspection_results",
    "defect_rate",
    "manufacturing_lead_time",
    "lead_time_days",
    "manufacturing_costs",
    "production_volumes",
    "supplier_name_original",
    "inspection_results_original",
    "defect_rates_original",
]
for c in requested_cols:
    if c not in df.columns:
        df[c] = pd.NA

# Reorder columns
out = df[requested_cols].copy()

# Save
out.to_csv(dst_path, index=False)
print(f"Saved: {dst_path}  (rows={len(out)})")

Saved: ..\data\derived\vendor_data.csv  (rows=2094)


In [7]:
import pandas as pd
from pathlib import Path

src = Path("../data/original/supply_chain_augmented.csv")
dst_dir = Path("../data/derived")
dst_dir.mkdir(parents=True, exist_ok=True)
dst = dst_dir / "products_data.csv"

df = pd.read_csv(src, parse_dates=["date"])

# Optional: these derived fields aren't needed for unique products, but harmless if you keep them
df["day"] = df["date"].dt.day
df["week"] = df["date"].dt.isocalendar().week.astype(int)
df["month"] = df["date"].dt.month
df["promotions"] = (df["discount_percent"] > 0).astype(int)

# Keep only unique product records
columns = ["sku_id", "product_type"]
products = (
    df[columns]
    .drop_duplicates()
    .sort_values(columns)  # stable, sorts by sku_id then product_type
    .reset_index(drop=True)
)

products.to_csv(dst, index=False)
print(f"Saved: {dst} (rows={len(products)})")

Saved: ..\data\derived\products_data.csv (rows=100)


In [8]:
import pandas as pd
from pathlib import Path

src = Path("../data/original/supply_chain_augmented.csv")
dst_dir = Path("../data/derived")
dst_dir.mkdir(parents=True, exist_ok=True)
dst = dst_dir / "warehouse_data.csv"

df = pd.read_csv(src, parse_dates=["date"])

columns = ["warehouse_id", "location"]

warehouses = (
    df[columns]
    .dropna(subset=columns)
    .drop_duplicates()
    .sort_values(columns)
    .reset_index(drop=True)
)

warehouses.to_csv(dst, index=False)
print(f"Saved: {dst} (rows={len(warehouses)})")

Saved: ..\data\derived\warehouse_data.csv (rows=4)


In [14]:
import sqlite3
import pandas as pd
from pathlib import Path

# Get the absolute path to the database
db_path = Path('../data/database/supply_chain.db').resolve()

# Verify the file exists
if not db_path.exists():
    print(f"Database not found at: {db_path}")
    # List files in the data directory to help debug
    data_dir = Path('data')
    print("\nFiles in data directory:")
    for f in data_dir.glob('**/*'):
        print(f" - {f.relative_to(data_dir.parent)}")
else:
    print(f"Found database at: {db_path}")
    
    # Connect to the database
    try:
        conn = sqlite3.connect(str(db_path))
        
        # List all tables
        tables = pd.read_sql("""
            SELECT name, sql 
            FROM sqlite_master 
            WHERE type='table' 
            AND name NOT LIKE 'sqlite_%'
            ORDER BY name
        """, conn)
        
        print("\nTables in the database:")
        for i, (table_name, _) in tables.iterrows():
            # Get row count for each table
            count = pd.read_sql(f"SELECT COUNT(*) as count FROM {table_name}", conn).iloc[0]['count']
            print(f" - {table_name}: {count} rows")
        
        # Show first 5 rows from each table
        for table_name in tables['name']:
            print(f"\nFirst 5 rows from {table_name}:")
            try:
                df = pd.read_sql(f"SELECT * FROM {table_name} LIMIT 5", conn)
                display(df)
            except Exception as e:
                print(f"  Could not read table {table_name}: {str(e)}")
        
    except Exception as e:
        print(f"Error connecting to database: {e}")
    finally:
        if 'conn' in locals():
            conn.close()

Found database at: E:\Learning\code-git\multi-agent-supplychain-ai\data\database\supply_chain.db

Tables in the database:
 - demand: 18000 rows
 - inventory: 17766 rows
 - product: 100 rows
 - purchase_order: 2094 rows
 - shipment: 16323 rows
 - vendor: 13477 rows
 - warehouse: 4 rows

First 5 rows from demand:


,date,sku_id,price,discount_percent,competitor_price,web_traffic,units_sold,created_at,updated_at
0,2025-05-20,SKU0,69.21,0.0,65.93,248,33,2025-11-15 18:21:03,2025-11-15 18:21:03
1,2025-05-21,SKU0,68.19,0.0,67.22,265,22,2025-11-15 18:21:03,2025-11-15 18:21:03
2,2025-05-22,SKU0,67.19,0.0,68.25,252,3,2025-11-15 18:21:03,2025-11-15 18:21:03
3,2025-05-23,SKU0,71.70,0.0,65.04,278,0,2025-11-15 18:21:03,2025-11-15 18:21:03
4,2025-05-24,SKU0,72.31,0.0,66.19,278,0,2025-11-15 18:21:03,2025-11-15 18:21:03



First 5 rows from inventory:


,warehouse_id,sku_id,stock_available,on_hand_qty,in_transit_qty,reorder_point,safety_stock
0,WH_Mumbai,SKU0,58,25,425,281,281
1,WH_Mumbai,SKU0,25,3,872,281,281
2,WH_Mumbai,SKU0,3,0,1322,281,281
3,WH_Mumbai,SKU0,0,0,1772,281,281
4,WH_Mumbai,SKU0,0,0,2222,281,281



First 5 rows from product:


...

,shipment_id,order_id,origin_lat,origin_lng,destination_lat,destination_lng,status,event_timestamp,estimated_delivery_date,actual_delivery_date,delay_hours,shipping_carrier,shipping_time_days,shipping_cost,created_at,updated_at
0,SHP-ORD-SKU0-20250520-08312c,ORD-SKU0-20250520-08312c,19.049875,72.887204,19.090697,72.811010,delivered,2025-05-20,2025-05-22,2025-05-24,48.0,Delhivery,1,5.00,2025-11-15 18:21:03,2025-11-15 18:21:03
1,SHP-ORD-SKU0-20250521-856e77,ORD-SKU0-20250521-856e77,19.073083,72.889002,19.091739,72.869679,delivered,2025-05-21,2025-05-23,2025-05-23,0.0,FedEx,1,3.01,2025-11-15 18:21:03,2025-11-15 18:21:03
2,SHP-ORD-SKU0-20250522-938610,ORD-SKU0-20250522-938610,19.076964,72.881891,19.029313,72.829497,delivered,2025-05-22,2025-05-26,2025-05-26,0.0,BlueDart,3,4.48,2025-11-15 18:21:03,2025-11-15 18:21:03
3,SHP-ORD-SKU0-20250528-cc1a24,ORD-SKU0-20250528-cc1a24,19.068167,72.874479,19.108843,72.812936,delivered,2025-05-28,2025-06-02,2025-06-02,0.0,BlueDart,4,6.82,2025-11-15 18:21:03,2025-11-15 18:21:03
4,SHP-ORD-SKU0-20250529-eb6cda,ORD-SKU0-20250529-eb6cda,19.087027,72.879448,19.026184,72.931916,delivered,2025-05-29,2025-06-01,2025-06-01,0.0,BlueDart,2,5.57,2025-11-15 18:21:03,2025-11-15 18:21:03



First 5 rows from vendor:


,vendor_id,supplier_name,defect_rate,lead_time_days,quality_score
0,Supplier 3,Supplier 3,0.0038,7,4.5
1,Supplier 3,Supplier 3,0.0613,7,4.5
2,Supplier 3,Supplier 3,0.0051,7,4.5
3,Supplier 3,Supplier 3,0.0484,7,4.5
4,Supplier 3,Supplier 3,0.0148,7,4.5



First 5 rows from warehouse:


,warehouse_id,location
0,WH_Mumbai,Mumbai
1,WH_Bengaluru,Bengaluru
2,WH_Chennai,Chennai
3,WH_Delhi,Delhi
